<a href="https://colab.research.google.com/github/korokke-3/aless/blob/main/aless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/alessデータ

/content/drive/MyDrive/alessデータ


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv('alessdata.csv')
test=pd.read_csv('alessdata2.csv')

In [ ]:
train.head()

,BPM,Key,Genre
0,103,E minor,samba
1,126,G minor,samba
2,94,C major,samba
3,96,E minor,samba
4,95,D minor,samba


In [ ]:
test.head()

,BPM,Key,Genre
0,133,D Major,samba
1,92,A Minor,samba
2,130,E Minor,samba
3,128,G Major,samba
4,117,A Major,samba


In [ ]:
x=train[['BPM','Key']]
y=train[['Genre']]

In [ ]:
x=pd.get_dummies(x,columns=['Key'])

In [ ]:
x.head()

,BPM,Key_A major,Key_A minor,Key_A♭ major,Key_B minor,Key_B♭ major,Key_B♭ minor,Key_C major,Key_C minor,Key_C# major,Key_D major,Key_D minor,Key_E major,Key_E minor,Key_E♭ major,Key_F major,Key_F# major,Key_F# minor,Key_G major,Key_G minor
0,103,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1,126,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,94,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,96,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4,95,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
# prompt: adapt standard scaler to BPM

from sklearn.preprocessing import StandardScaler

# Assuming 'BPM' is a column in your x_train and x_test DataFrames

scaler = StandardScaler()

# Fit the scaler on the training data only
scaler.fit(x_train[['BPM']])

# Transform both training and testing data
x_train['BPM'] = scaler.transform(x_train[['BPM']])
x_test['BPM'] = scaler.transform(x_test[['BPM']])

In [ ]:
# prompt: next step

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0)
model.fit(x_train, y_train.values.ravel())
model.score(x_test, y_test)

0.3333333333333333

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier()
knn.fit(x_train,y_train)
knn.score(x_test,y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.5

In [ ]:
from sklearn.svm import LinearSVC
svc=LinearSVC()
svc.fit(x_train,y_train)
svc.score(x_test,y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.2777777777777778

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
rf.score(x_test,y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.3333333333333333

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(x_train,y_train)
gb.score(x_test,y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.3888888888888889

In [ ]:
# prompt: Apply these to a test data set to test accuracy

# Preprocess the test data in the same way as the training data
x_test_final = test[['BPM', 'Key']]
x_test_final = pd.get_dummies(x_test_final, columns=['Key'])

# Ensure the same columns exist in both training and testing data
for col in x_train.columns:
    if col not in x_test_final.columns:
        x_test_final[col] = 0
for col in x_test_final.columns:
    if col not in x_train.columns:
        x_test_final = x_test_final.drop(col, axis=1)

#Scale BPM in test data using the scaler fitted on training data
x_test_final['BPM'] = scaler.transform(x_test_final[['BPM']])

#Align column order
x_test_final = x_test_final[x_train.columns]

# Make predictions using the trained models
y_pred_model = model.predict(x_test_final)
y_pred_knn = knn.predict(x_test_final)
y_pred_svc = svc.predict(x_test_final)
y_pred_rf = rf.predict(x_test_final)
y_pred_gb = gb.predict(x_test_final)

# Evaluate the models (you'll need a ground truth for the test data to calculate accuracy)
# Example: If you have the true labels in a column called 'Genre' in your test DataFrame
# from sklearn.metrics import accuracy_score
# accuracy_model = accuracy_score(test['Genre'], y_pred_model)
# print(f"RandomForestClassifier Accuracy on test data: {accuracy_model}")

# Similarly, calculate and print accuracy for knn, svc, rf, and gb
# ...

# To view predictions
print("Random Forest Predictions:\n", y_pred_model)
print("\nKNN Predictions:\n", y_pred_knn)
print("\nLinear SVC Predictions:\n", y_pred_svc)
print("\nRandom Forest (rf) Predictions:\n", y_pred_rf)
print("\nGradient Boosting Predictions:\n", y_pred_gb)

Random Forest Predictions:
 ['samba' 'bossanova' 'samba' 'samba' 'samba' 'jazz' 'samba' 'bossanova'
 'bossanova' 'samba' 'jazz' 'samba' 'samba' 'bossanova' 'bossanova'
 'bossanova' 'jazz' 'samba' 'samba' 'samba' 'bossanova' 'samba'
 'bossanova' 'bossanova' 'samba' 'samba' 'samba' 'samba' 'samba' 'samba'
 'jazz' 'samba' 'samba' 'bossanova' 'samba' 'bossanova' 'bossanova'
 'bossanova' 'samba' 'bossanova' 'samba' 'bossanova' 'bossanova'
 'bossanova' 'samba' 'bossanova' 'samba' 'bossanova' 'samba' 'samba'
 'bossanova' 'samba' 'samba' 'samba' 'samba' 'samba' 'bossanova'
 'bossanova' 'bossanova' 'samba' 'samba' 'samba' 'bossanova' 'samba'
 'bossanova' 'samba' 'samba' 'samba' 'samba' 'samba' 'jazz' 'bossanova'
 'samba' 'samba' 'bossanova' 'bossanova' 'samba' 'samba' 'bossanova'
 'samba' 'jazz' 'samba' 'bossanova' 'samba' 'jazz' 'samba' 'samba' 'samba'
 'samba' 'bossanova' 'bossanova' 'bossanova' 'bossanova' 'samba' 'samba'
 'samba' 'samba' 'bossanova' 'bossanova' 'samba' 'bossanova' 'bossanov

In [ ]:
# prompt: Check the percentage of correct answers.

from sklearn.metrics import accuracy_score

# Assuming 'Genre' is the column with true labels in your 'test' DataFrame
try:
    accuracy_model = accuracy_score(test['Genre'], y_pred_model)
    print(f"RandomForestClassifier Accuracy on test data: {accuracy_model}")

    accuracy_knn = accuracy_score(test['Genre'], y_pred_knn)
    print(f"KNN Accuracy on test data: {accuracy_knn}")

    accuracy_svc = accuracy_score(test['Genre'], y_pred_svc)
    print(f"LinearSVC Accuracy on test data: {accuracy_svc}")

    accuracy_rf = accuracy_score(test['Genre'], y_pred_rf)
    print(f"RandomForestClassifier (rf) Accuracy on test data: {accuracy_rf}")

    accuracy_gb = accuracy_score(test['Genre'], y_pred_gb)
    print(f"GradientBoostingClassifier Accuracy on test data: {accuracy_gb}")

except KeyError:
    print("Error: 'Genre' column not found in the test DataFrame.  Please ensure your test data includes a 'Genre' column with the correct labels.")
except Exception as e:
    print(f"An error occurred during accuracy calculation: {e}")

RandomForestClassifier Accuracy on test data: 0.3641791044776119
KNN Accuracy on test data: 0.3402985074626866
LinearSVC Accuracy on test data: 0.3850746268656716
RandomForestClassifier (rf) Accuracy on test data: 0.3761194029850746
GradientBoostingClassifier Accuracy on test data: 0.373134328358209
